In [13]:
import pandas as pd

df = pd.DataFrame(pd.read_csv('../data/SanFrancisco_salary_dirty.csv'),
                  columns=['JobTitle', 'Department', 'BasePay', 'OvertimePay', 'OtherPay', 'Benefits', 'TotalPay'])

df.describe()

,BasePay,OvertimePay,OtherPay,Benefits,TotalPay
count,111881.000000,111881.000000,111881.000000,111881.000000,111881.000000
mean,67208.464335,5266.537354,3655.130550,25114.682525,76130.132232
std,43417.695515,11764.757612,8038.765165,15372.746237,51269.839386
min,-166.010000,-0.010000,0.000000,-13.800000,0.000000
25%,33644.660156,0.000000,0.000000,12145.089844,37041.430000
50%,65547.040000,0.000000,820.840000,28698.490234,72630.700000
75%,95229.030000,4865.720000,4307.970215,35608.770000,107533.480000
max,319275.010000,220909.480000,342802.630000,96570.660000,471952.640000


In [17]:
def get_total_pay(df: pd.DataFrame, row):
    return df.loc['BasePay'] + df.loc['OvertimePay'] + df.loc['OtherPay'] + df.loc['Benefits']


df = pd.DataFrame(pd.read_csv('../data/SanFrancisco_salary_dirty.csv'),
                  columns=['JobTitle', 'Department', 'BasePay', 'OvertimePay', 'OtherPay', 'Benefits', 'TotalPay'])

for x in df.index:
    if df.loc[x, 'OvertimePay'] < 0:
        df.loc[x, 'OvertimePay'] = 0

for x in df.index:
    if df.loc[x, 'Benefits'] < 0:
        df.loc[x, 'Benefits'] = 0


all_depts = df['Department'].unique()

for x in df.index:
    total_pay = df.loc[x, 'TotalPay']
    if total_pay <= 0 or total_pay != get_total_pay(df, x):
        df.loc[x, 'TotalPay'] = get_total_pay(df, x)

print(df.describe())
df.to_csv('../data/SanFrancisco_salary_clean.csv')


['Fire' 'Other' 'Not dep' 'Medicine' 'Police']
['Fire' 'Other' 'Not dep' 'Medicine' 'Police']
['Fire' 'Other' 'Not dep' 'Medicine' 'Police']
['Fire' 'Other' 'Not dep' 'Medicine' 'Police']
['Fire' 'Other' 'Not dep' 'Medicine' 'Police']
['Fire' 'Other' 'Not dep' 'Medicine' 'Police']
['Fire' 'Other' 'Not dep' 'Medicine' 'Police']
['Fire' 'Other' 'Not dep' 'Medicine' 'Police']
['Fire' 'Other' 'Not dep' 'Medicine' 'Police']
['Fire' 'Other' 'Not dep' 'Medicine' 'Police']
['Fire' 'Other' 'Not dep' 'Medicine' 'Police']
['Fire' 'Other' 'Not dep' 'Medicine' 'Police']
['Fire' 'Other' 'Not dep' 'Medicine' 'Police']
['Fire' 'Other' 'Not dep' 'Medicine' 'Police']
['Fire' 'Other' 'Not dep' 'Medicine' 'Police']
['Fire' 'Other' 'Not dep' 'Medicine' 'Police']
['Fire' 'Other' 'Not dep' 'Medicine' 'Police']
['Fire' 'Other' 'Not dep' 'Medicine' 'Police']
['Fire' 'Other' 'Not dep' 'Medicine' 'Police']
['Fire' 'Other' 'Not dep' 'Medicine' 'Police']
['Fire' 'Other' 'Not dep' 'Medicine' 'Police']
['Fire' 'Othe

KeyboardInterrupt: 